In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
from tqdm import tqdm
from keras.applications.vgg16 import VGG16
import cv2
import math
import os
from glob import glob
from scipy import stats as s
from sklearn.externals import joblib 

Using TensorFlow backend.
C:\Users\HH TRADERS\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
base_model = VGG16(weights='imagenet', include_top=False)

In [3]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [4]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(51, activation='softmax'))

In [5]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)              

In [6]:
# loading the trained weights
model.load_weights("../Models/weightVGG16.hdf5")

In [7]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [8]:
location='../data/test'
classes=[]
videos=[]
test=pd.DataFrame()
for foldername in sorted(os.listdir(location)):
    classes.append(foldername)
    for files in sorted(os.listdir(location+'/'+foldername)):
        videos.append(foldername+'/'+files)
test['video_name']=videos
print(classes)
test_videos=test['video_name']
test.head()

['brush_hair', 'cartwheel', 'catch', 'chew', 'clap', 'climb', 'climb_stairs', 'dive', 'draw_sword', 'dribble', 'drink', 'eat', 'fall_floor', 'fencing', 'flic_flac', 'golf', 'handstand', 'hit', 'hug', 'jump', 'kick', 'kick_ball', 'kiss', 'laugh', 'pick', 'pour', 'pullup', 'punch', 'push', 'pushup', 'ride_bike', 'ride_horse', 'run', 'shake_hands', 'shoot_ball', 'shoot_bow', 'shoot_gun', 'sit', 'situp', 'smile', 'smoke', 'somersault', 'stand', 'swing_baseball', 'sword', 'sword_exercise', 'talk', 'throw', 'turn', 'walk', 'wave']


,video_name
0,brush_hair/April_09_brush_hair_u_nm_np1_ba_goo...
1,brush_hair/April_09_brush_hair_u_nm_np1_ba_goo...
2,brush_hair/April_09_brush_hair_u_nm_np1_ba_goo...
3,brush_hair/Blonde_being_brushed_brush_hair_f_n...
4,brush_hair/Blonde_being_brushed_brush_hair_u_c...


In [9]:
# creating the tags
train = pd.read_csv('../data/train.csv')
y = train['class']
y = pd.get_dummies(y)

In [10]:
# creating two lists to store predicted and actual tags
predict = []
actual = []

# for loop to extract frames from each test video
for i in tqdm(range(test_videos.shape[0])):
    count = 0
    videoFile = test_videos[i]
    print('../data/test/'+videoFile)

  0%|          | 0/1530 [00:00<?, ?it/s]

../data/test/brush_hair/April_09_brush_hair_u_nm_np1_ba_goo_0.avi
../data/test/brush_hair/April_09_brush_hair_u_nm_np1_ba_goo_1.avi
../data/test/brush_hair/April_09_brush_hair_u_nm_np1_ba_goo_2.avi
../data/test/brush_hair/Blonde_being_brushed_brush_hair_f_nm_np2_ri_med_0.avi
../data/test/brush_hair/Blonde_being_brushed_brush_hair_u_cm_np2_ri_med_1.avi
../data/test/brush_hair/Brushing_Hair_with_Beth_brush_hair_h_nm_np1_le_goo_0.avi
../data/test/brush_hair/Brushing_Hair_with_Beth_brush_hair_u_nm_np1_fr_goo_1.avi
../data/test/brush_hair/Brushing_Hair_with_Beth_brush_hair_u_nm_np1_fr_goo_2.avi
../data/test/brush_hair/Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_brush_hair_h_cm_np1_fr_goo_0.avi
../data/test/brush_hair/Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_brush_hair_h_cm_np1_le_goo_1.avi
../data/test/brush_hair/Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED!!!!_brush_hair_h_cm_np1_le_goo_2.avi
../data/test/brush_hair/Brushing_my_waist_lenth_hair_brush_hair_u_nm_np1_ba_goo_0.avi
../data/tes

  2%|▏         | 38/1530 [00:00<00:04, 359.38it/s]

../data/test/cartwheel/Handstandk_nig_cartwheel_f_cm_np1_ri_med_4.avi
../data/test/cartwheel/How_to_Do_an_Aerial_-_Aerial_vs__Cartwheel_cartwheel_f_cm_np1_le_med_0.avi
../data/test/cartwheel/How_to_Do_an_Aerial_-_Aerial_vs__Cartwheel_cartwheel_f_cm_np1_ri_med_1.avi
../data/test/cartwheel/How_to_do_a_Cartwheel_cartwheel_f_nm_np1_le_med_1.avi
../data/test/cartwheel/How_to_do_a_Cartwheel_cartwheel_f_nm_np1_ri_med_0.avi
../data/test/cartwheel/Jessica_Heinz_-_Leverkusen_Cup_2006__floor_routine_cartwheel_f_cm_np1_le_med_2.avi
../data/test/cartwheel/Jessica_Heinz_-_Leverkusen_Cup_2006__floor_routine_cartwheel_f_cm_np1_le_med_5.avi
../data/test/cartwheel/Jessica_Heinz_-_Leverkusen_Cup_2006__floor_routine_cartwheel_f_cm_np1_ri_med_0.avi
../data/test/cartwheel/Jessica_Heinz_-_Leverkusen_Cup_2006__floor_routine_cartwheel_f_cm_np1_ri_med_4.avi
../data/test/cartwheel/Mirco_turnt_die_5_2_LAD___Schraube_rw_cartwheel_f_cm_np1_le_med_2.avi
../data/test/cartwheel/Mirco_turnt_die_5_2_LAD___Schraube_rw_ca

  5%|▍         | 69/1530 [00:00<00:04, 338.20it/s]

...

 96%|█████████▋| 1474/1530 [00:03<00:00, 386.54it/s]

../data/test/walk/AmericanGangster_walk_f_nm_np1_ba_med_30.avi
../data/test/walk/AmericanGangster_walk_f_nm_np1_fr_med_73.avi
../data/test/walk/AmericanGangster_walk_u_cm_np1_ba_med_37.avi
../data/test/walk/AmericanGangster_walk_u_cm_np1_ri_med_24.avi
../data/test/walk/BATMAN_BEGINS_walk_f_cm_np1_fr_med_4.avi
../data/test/walk/CastAway1_walk_f_nm_np1_fr_med_12.avi
../data/test/walk/Juno_walk_f_nm_np1_fr_med_1.avi
../data/test/walk/Oceans11_walk_u_cm_np1_fr_med_1.avi
../data/test/walk/Oceans12_walk_u_cm_np1_fr_med_17.avi
../data/test/walk/Oceans12_walk_u_cm_np1_ri_med_1.avi
../data/test/walk/Return_of_the_King_11_walk_f_cm_np2_ri_med_3.avi
../data/test/walk/SweeneyTodd_walk_f_cm_np1_le_med_7.avi
../data/test/walk/SweeneyTodd_walk_f_nm_np1_fr_med_29.avi
../data/test/walk/TheBigBangS01E02TheBigBranHypothesis_walk_f_nm_np1_le_med_3.avi
../data/test/walk/TheBoondockSaints_walk_u_cm_np1_fr_med_33.avi
../data/test/walk/TheBoondockSaints_walk_u_cm_np1_fr_med_49.avi
../data/test/walk/TheBoondoc

100%|█████████▉| 1523/1530 [00:03<00:00, 412.46it/s]

../data/test/wave/OldSchool_wave_f_cm_np1_fr_med_13.avi
../data/test/wave/OldSchool_wave_u_nm_np1_fr_med_24.avi
../data/test/wave/likebeckam_wave_u_cm_np1_fr_med_6.avi
../data/test/wave/likebeckam_wave_u_cm_np1_le_med_20.avi
../data/test/wave/prideandprejudice1_wave_f_nm_np1_ri_med_14.avi
../data/test/wave/veoh_harold_and_kumar_wave_u_nm_np1_fr_goo_5.avi
../data/test/wave/wave_and_say_hi_wave_u_nm_np1_fr_med_0.avi


100%|██████████| 1530/1530 [00:03<00:00, 391.99it/s]


In [12]:
# creating two lists to store predicted and actual tags
predict = []
actual = []

# for loop to extract frames from each test video
for i in tqdm(range(test_videos.shape[0])):
    count = 0
    videoFile = test_videos[i]
    #print('../data/test/'+videoFile)
    cap = cv2.VideoCapture('../data/test/'+videoFile.split(' ')[0].split('/')[0]+'/'+videoFile.split(' ')[0].split('/')[1])   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    x=1
    # removing all other files from the temp folder
    files = glob('../data/temp/*')
    for f in files:
        os.remove(f)
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames of this particular video in temp folder
#            print('../data/temp/' + "_frame%d.jpg" % count)
            filename ='../data/temp/' + "_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()
    
    # reading all the frames from temp folder
    images = glob("../data/temp/*.jpg")
    
    prediction_images = []
    for i in range(len(images)):
        img = image.load_img(images[i], target_size=(224,224,3))
        img = image.img_to_array(img)
        img = img/255
        prediction_images.append(img)
        
    # converting all the frames for a test video into numpy array
    prediction_images = np.array(prediction_images)
    # extracting features using pre-trained model
    prediction_images = base_model.predict(prediction_images)
    # converting features in one dimensional array
    prediction_images = prediction_images.reshape(prediction_images.shape[0], 7*7*512)
    # predicting tags for each array
    prediction = model.predict_classes(prediction_images)
    # appending the mode of predictions in predict list to assign the tag to the video
    predict.append(y.columns.values[s.mode(prediction)[0][0]])
    # appending the actual tag of the video
    actual.append(videoFile.split('/')[0])



 20%|██        | 312/1530 [18:32<1:09:15,  3.41s/it]


 41%|████      | 630/1530 [31:13<25:19,  1.69s/it]


 62%|██████▏   | 948/1530 [48:32<47:53,  4.94s/it]


 82%|████████▏ | 1259/1530 [1:05:45<12:10,  2.70s/it]


100%|██████████| 1530/1530 [1:18:50<00:00,  2.31s/it]

In [13]:
# Save the model as a pickle in a file 
joblib.dump(predict, '../Pickle/VGG16_predict.pkl') 
joblib.dump(actual, '../Pickle/VGG16_actual.pkl')

['../Pickle/VGG16_actual.pkl']

In [14]:
# Load the model from the file 
history = joblib.load('../Pickle/VGG16_predict.pkl') 
history = joblib.load('../Pickle/VGG16_actual.pkl') 

In [15]:
# checking the accuracy of the predicted tags
from sklearn.metrics import accuracy_score
accuracy_score(predict, actual)*100

17.38562091503268

In [16]:
from sklearn import metrics
print(metrics.classification_report(actual,predict))

C:\Users\HH TRADERS\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                precision    recall  f1-score   support

    brush_hair       0.08      0.23      0.11        30
     cartwheel       0.10      0.17      0.12        30
         catch       0.17      0.07      0.10        30
          chew       0.12      0.23      0.16        30
          clap       0.02      0.03      0.03        30
         climb       0.28      0.53      0.36        30
  climb_stairs       0.03      0.07      0.04        30
          dive       0.00      0.00      0.00        30
    draw_sword       0.04      0.03      0.04        30
       dribble       0.29      0.53      0.37        30
         drink       0.06      0.13      0.08        30
           eat       0.09      0.17      0.12        30
    fall_floor       1.00      0.03      0.06        30
       fencing       0.00      0.00      0.00        30
     flic_flac       0.00      0.00      0.00        30
          golf       0.27      0.73      0.40        30
     handstand       0.10      0.13      0.11  

In [17]:
cm=metrics.confusion_matrix(actual,predict)
print(cm)

[[7 0 0 ... 0 0 0]
 [0 5 0 ... 0 0 0]
 [1 1 2 ... 0 0 0]
 ...
 [2 0 0 ... 0 0 0]
 [3 0 1 ... 0 0 0]
 [3 0 0 ... 0 0 0]]


In [18]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(9,9))
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square=True, cmap="Blues_r");
plt.ylabel("Actual label");
plt.xlabel("Predicted label")
all_sample_title="Accuracy score: {0}".format(metrics.accuracy_score(predict, actual)*100)
plt.title(all_sample_title,size=15)

Text(0.5, 1, 'Accuracy score: 17.38562091503268')